# Minimal Fantasy Football Draft Board

Everything you need for draft day in one simple notebook. No complexity, just results.

In [ ]:
# Cell 1: Setup and Data Loading
import sys
sys.path.append('../src')

import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.colors import ListedColormap
import pandas as pd
import numpy as np
import yaml
from draft_engine import DraftState, DraftIntelligence, Player
import warnings
warnings.filterwarnings('ignore')

# Load configuration and data
with open('../config/league-config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Load player data and standardize columns
players_df = pd.read_csv('../draft_cheat_sheet.csv')

# Map columns to expected format (fix column names to match actual CSV)
column_mapping = {
    'Player': 'UNNAMED:_0_LEVEL_0_PLAYER',
    'Position': 'POSITION', 
    'Custom_VBD': 'FANTASY_PTS',
    'Team': 'TEAM',
    'Bye': 'BYE_WEEK'
}

for old_col, new_col in column_mapping.items():
    if old_col in players_df.columns:
        players_df[new_col] = players_df[old_col]

# Ensure required columns exist
if 'FANTASY_PTS' not in players_df.columns and 'Custom_VBD' in players_df.columns:
    players_df['FANTASY_PTS'] = players_df['Custom_VBD']

print(f"✅ Loaded {len(players_df)} players")
print(f"✅ League: {config['league_name']} ({config['basic_settings']['teams']} teams)")
print(f"✅ Columns available: {list(players_df.columns[:5])}")

# Ask for user's draft position  
USER_TEAM_ID = 1  # Change this to your draft position (1-14)
USER_DRAFT_POSITION = 1  # Change this to your draft position

In [6]:
# Cell 2: Initialize Draft System
draft_state = DraftState(config, USER_TEAM_ID, USER_DRAFT_POSITION)
intelligence = DraftIntelligence(config, players_df)

# Position colors for visualization
POSITION_COLORS = {
    'QB': '#FF6B6B',   # Red
    'RB': '#4ECDC4',   # Teal
    'WR': '#45B7D1',   # Blue
    'TE': '#96CEB4',   # Green
    'K': '#DDA0DD',    # Plum
    'DST': '#FFEAA7',  # Yellow
    'DEF': '#FFEAA7'   # Yellow
}

print(f"🎯 You are Team {USER_TEAM_ID} - '{draft_state.teams[USER_TEAM_ID].name}'")
print(f"📍 Draft Position: {USER_DRAFT_POSITION}")
print(f"🏈 Ready to draft!")

🎯 You are Team 1 - 'Bam Bam Kam'
📍 Draft Position: 1
🏈 Ready to draft!


In [7]:
# Cell 3: Draft Manager Class
class MinimalDraftManager:
    def __init__(self, draft_state, intelligence):
        self.state = draft_state
        self.intelligence = intelligence
        self.draft_history = []
        
        # Create widgets
        self.search_input = widgets.Text(
            placeholder='Type player name...',
            description='Search:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        
        self.player_dropdown = widgets.Dropdown(
            description='Player:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        
        self.draft_button = widgets.Button(
            description='DRAFT PLAYER',
            button_style='success',
            layout=widgets.Layout(width='120px')
        )
        
        self.auto_button = widgets.Button(
            description='AUTO PICK',
            button_style='warning',
            layout=widgets.Layout(width='100px')
        )
        
        self.undo_button = widgets.Button(
            description='UNDO',
            button_style='danger',
            layout=widgets.Layout(width='80px')
        )
        
        # Output areas
        self.board_output = widgets.Output()
        self.recommendations_output = widgets.Output()
        self.roster_output = widgets.Output()
        
        # Wire up events
        self.search_input.observe(self.on_search_change, names='value')
        self.draft_button.on_click(self.on_draft_click)
        self.auto_button.on_click(self.on_auto_click)
        self.undo_button.on_click(self.on_undo_click)
        
        # Create layout
        controls = widgets.VBox([
            widgets.HTML('<h3>🏈 Draft Controls</h3>'),
            self.search_input,
            self.player_dropdown,
            widgets.HBox([self.draft_button, self.auto_button, self.undo_button])
        ])
        
        recommendations = widgets.VBox([
            widgets.HTML('<h3>🤖 AI Recommendations</h3>'),
            self.recommendations_output
        ])
        
        roster_panel = widgets.VBox([
            widgets.HTML('<h3>👥 Your Roster</h3>'),
            self.roster_output
        ])
        
        self.layout = widgets.VBox([
            widgets.HBox([controls, recommendations, roster_panel]),
            widgets.HTML('<h3>📊 Draft Board</h3>'),
            self.board_output
        ])
        
        # Initialize
        self.refresh_all()
    
    def on_search_change(self, change):
        """Handle search input changes"""
        search_term = change['new'].lower()
        available_players = self.intelligence.get_available_players_with_scores(self.state)
        
        # Filter by search term
        if search_term:
            filtered = [(p, s) for p, s in available_players 
                       if search_term in p.name.lower()]
        else:
            filtered = available_players[:50]  # Show top 50 if no search
        
        # Update dropdown options
        options = [(f"{p.name} ({p.position}) - VBD:{p.vbd:.0f}", p.id) 
                  for p, s in filtered[:20]]  # Top 20 matches
        self.player_dropdown.options = options
    
    def on_draft_click(self, button):
        """Handle draft button click"""
        if not self.player_dropdown.value:
            return
        
        player_id = self.player_dropdown.value
        player = self.intelligence.players_dict[player_id]
        team_on_clock = self.state.get_team_on_clock()
        
        self.state.make_pick(team_on_clock, player)
        self.draft_history.append({'team': team_on_clock, 'player': player})
        self.refresh_all()
    
    def on_auto_click(self, button):
        """Handle auto pick button click"""
        recommendations = self.intelligence.get_recommendations(self.state, 1)
        if recommendations:
            player = recommendations[0][0]
            team_on_clock = self.state.get_team_on_clock()
            
            self.state.make_pick(team_on_clock, player)
            self.draft_history.append({'team': team_on_clock, 'player': player})
            self.refresh_all()
    
    def on_undo_click(self, button):
        """Handle undo button click"""
        if self.draft_history:
            last_pick = self.draft_history.pop()
            self.state.undo_pick(last_pick['player'])
            self.refresh_all()
    
    def refresh_all(self):
        """Refresh all components"""
        self.refresh_board()
        self.refresh_recommendations()
        self.refresh_roster()
        self.on_search_change({'new': self.search_input.value})
    
    def refresh_recommendations(self):
        """Refresh AI recommendations panel"""
        with self.recommendations_output:
            clear_output()
            
            recommendations = self.intelligence.get_recommendations(self.state, 5)
            if not recommendations:
                print("No recommendations available")
                return
            
            print(f"📋 Top Picks (Pick #{self.state.current_pick})\n")
            
            for i, (player, score, reasoning) in enumerate(recommendations, 1):
                # Color coding by position
                pos_color = POSITION_COLORS.get(player.position, '#95A5A6')
                
                print(f"{i}. {player.name} ({player.position})")
                print(f"   VBD: {player.vbd:.0f} | Score: {score:.1f}")
                print(f"   💡 {reasoning}")
                print()
    
    def refresh_roster(self):
        """Refresh user roster panel"""
        with self.roster_output:
            clear_output()
            
            roster = self.state.get_user_roster()
            pick_info = self.state.get_current_pick_info()
            
            print(f"Team: {self.state.teams[self.state.user_team_id].name}")
            print(f"Next Pick: #{self.state.get_your_next_pick()}")
            print(f"Picks Until Turn: {self.state.get_picks_until_your_turn()}\n")
            
            # Show roster by position
            for position in ['QB', 'RB', 'WR', 'TE', 'K', 'DST']:
                players_at_pos = roster.get(position, [])
                need_count = self.state.teams[self.state.user_team_id].get_starter_slots_needed(position)
                
                status = "✅" if len(players_at_pos) >= need_count else "❌" 
                print(f"{status} {position} ({len(players_at_pos)}/{need_count}):")
                
                for player in players_at_pos:
                    print(f"   • {player.name} (VBD: {player.vbd:.0f})")
                print()
    
    def draw_player_tile(self, ax, x, y, player, pick_num, adp=None):
        """Draw rich player tile with name, position, VBD, and value indicators"""
        # Base color by position
        color = POSITION_COLORS.get(player.position, '#95A5A6')
        
        # Value/reach indicator - green if good value, red if reach
        border_color = '#27AE60'  # Default green
        if adp and pick_num > adp:
            border_color = '#27AE60'  # Good value (drafted later than ADP)
        elif adp and pick_num < adp - 20:  # Significant reach
            border_color = '#E74C3C'  # Red for reach
        
        # Draw main tile as rectangle for more text space
        tile = patches.Rectangle((x-0.45, y-0.35), 0.9, 0.7, 
                               facecolor=color, edgecolor=border_color, linewidth=2)
        ax.add_patch(tile)
        
        # Player name (first name + last initial)
        name_parts = player.name.split()
        if len(name_parts) >= 2:
            display_name = f"{name_parts[0]} {name_parts[-1][0]}."
        else:
            display_name = name_parts[0][:8]  # Truncate if long single name
        
        # Multi-line text: Name, Position, VBD
        ax.text(x, y+0.1, display_name, ha='center', va='center', 
               fontsize=7, color='white', weight='bold')
        ax.text(x, y-0.05, player.position, ha='center', va='center', 
               fontsize=8, color='white', weight='bold')
        ax.text(x, y-0.2, f"VBD:{player.vbd:.0f}", ha='center', va='center', 
               fontsize=6, color='white')
    
    def refresh_board(self):
        with self.board_output:
            clear_output()
            
            # Create draft board grid
            fig, ax = plt.subplots(figsize=(12, 10))
            
            teams = self.state.config['basic_settings']['teams']
            rounds = 16  # Total rounds
            
            # Draw grid
            for round_num in range(1, rounds + 1):
                for pick_in_round in range(1, teams + 1):
                    # Calculate actual pick number
                    if round_num % 2 == 1:  # Odd round
                        team_id = pick_in_round
                    else:  # Even round (snake)
                        team_id = teams - pick_in_round + 1
                    
                    pick_num = (round_num - 1) * teams + pick_in_round
                    
                    # Check if pick has been made
                    pick = self.state.get_pick_by_number(pick_num)
                    
                    x = pick_in_round - 0.5
                    y = rounds - round_num + 0.5
                    
                    if pick:
                        # Draw rich player tile
                        self.draw_player_tile(ax, x, y, pick.player, pick_num, pick.player.adp)
                    else:
                        # Draw empty slot
                        empty_tile = patches.Rectangle((x-0.45, y-0.35), 0.9, 0.7, 
                                                     facecolor='#ECF0F1', edgecolor='#BDC3C7', linewidth=1)
                        ax.add_patch(empty_tile)
                        
                        # Highlight current pick
                        if pick_num == self.state.current_pick:
                            highlight = patches.Rectangle((x-0.5, y-0.4), 1, 0.8, 
                                                        facecolor='none', edgecolor='#E74C3C', 
                                                        linewidth=3, linestyle='--')
                            ax.add_patch(highlight)
                            ax.text(x, y, "ON\nCLOCK", ha='center', va='center', 
                                   fontsize=8, color='#E74C3C', weight='bold')
                    
                    # Highlight user's picks with special border
                    if pick and pick.team_id == self.state.user_team_id:
                        user_highlight = patches.Rectangle((x-0.5, y-0.4), 1, 0.8, 
                                                          facecolor='none', edgecolor='#F39C12', 
                                                          linewidth=3, linestyle='-')
                        ax.add_patch(user_highlight)
            
            # Setup axes
            ax.set_xlim(0, teams)
            ax.set_ylim(0, rounds)
            ax.set_aspect('equal')
            
            # Labels
            ax.set_xticks(range(teams))
            ax.set_xticklabels([f"T{i+1}" for i in range(teams)], fontsize=9)
            ax.set_yticks(range(rounds))
            ax.set_yticklabels([f"R{rounds-i}" for i in range(rounds)], fontsize=9)
            
            ax.set_xlabel('Team', fontsize=11, weight='bold')
            ax.set_ylabel('Round', fontsize=11, weight='bold')
            ax.set_title(f'Fantasy Draft Board - Pick {self.state.current_pick}', 
                        fontsize=14, weight='bold')
            
            # Enhanced legend with value indicators
            legend_elements = [patches.Patch(facecolor=color, label=pos) 
                              for pos, color in POSITION_COLORS.items()]
            # Add value indicator legend
            legend_elements.append(patches.Patch(facecolor='none', edgecolor='#27AE60', 
                                               linewidth=2, label='Good Value'))
            legend_elements.append(patches.Patch(facecolor='none', edgecolor='#E74C3C', 
                                               linewidth=2, label='Reach'))
            legend_elements.append(patches.Patch(facecolor='none', edgecolor='#F39C12', 
                                               linewidth=2, label='Your Pick'))
            
            ax.legend(handles=legend_elements, loc='upper left', bbox_to_anchor=(1, 1), 
                     fontsize=9, title="Positions & Values", title_fontsize=10)
            
            plt.tight_layout()
            plt.show()

In [8]:
# Cell 4: Initialize and Display Interface
manager = MinimalDraftManager(draft_state, intelligence)
display(manager.layout)

## Quick Actions for Testing

In [9]:
# Cell 5: Quick Draft Functions (Optional - for testing)
def simulate_picks(num_picks=5):
    """Simulate draft picks for testing"""
    print(f"🤖 Simulating {num_picks} picks...\n")
    
    for i in range(num_picks):
        if draft_state.current_pick > 14 * 16:  # Max picks
            break
            
        # Get best available
        recs = intelligence.get_recommendations(draft_state, 1)
        if not recs:
            break
            
        player = recs[0][0]
        team_id = draft_state.get_team_on_clock()
        draft_state.make_pick(team_id, player)
        manager.draft_history.append({'team': team_id, 'player': player})
        
        print(f"Pick {draft_state.current_pick - 1}: {draft_state.teams[team_id].name} selects {player.name} ({player.position})")
    
    manager.refresh_all()
    print("\n✅ Simulation complete!")

# Quick action buttons
sim_5 = widgets.Button(description='Simulate 5 Picks', button_style='info')
sim_round = widgets.Button(description='Simulate Round', button_style='info')
reset = widgets.Button(description='Reset Draft', button_style='danger')

def on_sim_5(b):
    simulate_picks(5)

def on_sim_round(b):
    simulate_picks(14)

def on_reset(b):
    global draft_state, manager
    draft_state = DraftState(config, USER_TEAM_ID, USER_DRAFT_POSITION)
    manager = MinimalDraftManager(draft_state, intelligence)
    display(manager.layout)
    print("🔄 Draft reset!")

sim_5.on_click(on_sim_5)
sim_round.on_click(on_sim_round)
reset.on_click(on_reset)

display(widgets.HBox([sim_5, sim_round, reset]))

## 📝 Draft Day Instructions

### Before the Draft:
1. Update `USER_TEAM_ID` and `USER_DRAFT_POSITION` in Cell 1
2. Run all cells to initialize

### During the Draft:
1. **Search**: Type player name to filter
2. **Select**: Choose from dropdown
3. **Draft**: Click DRAFT PLAYER
4. **Auto**: Let AI pick for you
5. **Undo**: Fix mistakes immediately

### Key Features:
- 🎯 **Smart Recommendations**: AI-powered picks with reasoning
- ⚠️ **Scarcity Alerts**: Know when positions are running out
- 🏈 **Visual Board**: See the entire draft at a glance
- 📊 **Your Roster**: Track your team needs in real-time

### Color Legend:
- 🔴 QB (Red)
- 🟦 RB (Teal)
- 🔵 WR (Blue)  
- 🟢 TE (Green)
- 🟡 DEF (Yellow)
- 🟣 K (Purple)

**Good luck with your draft!** 🏆